In [1]:
import pandas as pd


In [3]:
# Load the dataset
file_path = "C:/Users/patha/OneDrive/Desktop/internships/vervebridge/task 3/Football-Scenarios-DFE-832307.csv"
df = pd.read_csv(file_path)

In [4]:
# Display the first few rows to inspect the data
df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,antecedent,antecedent:confidence,orig_antecedent,antecedent_gold,option1,option2,option3,option4,option5
0,831005673,False,finalized,5,11/20/15 20:20,kick a field goal,0.8092,It is first down and 10. The ball is on your o...,NaN,punt,kick a field goal,run,pass,kneel down
1,831005674,False,finalized,5,11/18/15 21:59,kick a field goal,1.0000,It is second down and inches. The ball is on y...,NaN,punt,kick a field goal,run,pass,kneel down
2,831005675,False,finalized,5,11/20/15 22:43,kick a field goal,0.6211,It is second down and inches. The ball is on y...,NaN,punt,kick a field goal,run,pass,kneel down
3,831005676,False,finalized,5,11/19/15 7:41,kick a field goal,0.8073,It is second down and inches. The ball is on y...,NaN,punt,kick a field goal,run,pass,kneel down
4,831005677,False,finalized,5,11/21/15 8:01,kick a field goal,1.0000,It is second down and inches. The ball is on y...,NaN,punt,kick a field goal,run,pass,kneel down


In [5]:
# Dropping columns that are not useful for analysis or Power BI visualization
columns_to_drop = ['_unit_id', '_golden', '_unit_state', '_trusted_judgments', '_last_judgment_at']
df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

In [6]:
# Renaming columns for better readability in Power BI
df.rename(columns={
    'antecedent': 'Decision',
    'antecedent:confidence': 'Decision_Confidence',
    'orig_antecedent': 'Scenario',
    'antecedent_gold': 'Gold_Standard'
}, inplace=True)

In [7]:
# Check the unique values in the decision confidence column to verify conversion is accurate
df['Decision_Confidence'] = pd.to_numeric(df['Decision_Confidence'], errors='coerce')

In [8]:
# Fill NaN values in categorical columns with 'Unknown' or 0 for numeric fields
df.fillna({
    'Gold_Standard': 'Unknown', 
    'Decision_Confidence': 0
}, inplace=True)

In [9]:
#One-hot encoding options if they represent distinct categories
df = pd.concat([df, pd.get_dummies(df[['option1', 'option2', 'option3', 'option4', 'option5']].stack()).groupby(level=0).max()], axis=1)
df.drop(columns=['option1', 'option2', 'option3', 'option4', 'option5'], inplace=True)

In [10]:
# Check the dataset info to understand its structure
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3730 entries, 0 to 3729
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Decision             3730 non-null   object 
 1   Decision_Confidence  3730 non-null   float64
 2   Scenario             3730 non-null   object 
 3   Gold_Standard        3730 non-null   object 
 4    kick a field goal   3730 non-null   bool   
 5    kneel down          3730 non-null   bool   
 6    pass                3730 non-null   bool   
 7    punt                3730 non-null   bool   
 8    run                 3730 non-null   bool   
dtypes: bool(5), float64(1), object(3)
memory usage: 134.9+ KB


In [11]:
# Splitting a column if 'Time Left' has both minutes and seconds in one field
if 'Time Left' in df.columns:
    # Split 'Time Left' column into minutes and seconds
    time_split = df['Time Left'].str.extract(r'(?P<Minutes>\d+)\s*mins?\s*(?P<Seconds>\d+)?')
    time_split['Minutes'] = pd.to_numeric(time_split['Minutes'], errors='coerce')
    time_split['Seconds'] = pd.to_numeric(time_split['Seconds'], errors='coerce').fillna(0)
    # Drop original 'Time Left' and join new columns
    df = df.drop(columns=['Time Left']).join(time_split)


In [12]:
#Convert 'Down' to a numeric data type if it's categorical
if 'Down' in df.columns:
    df['Down'] = pd.to_numeric(df['Down'], errors='coerce')

In [13]:
# Convert other relevant columns to numeric types if needed
numeric_columns = ['Yard Line', 'Score Difference']  # Adjust based on actual columns
for col in numeric_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

In [14]:
# Final check for missing values and clean up if needed
df.fillna(0, inplace=True)


In [20]:
# Export the cleaned data to a new CSV file ready for Power BI
cleaned_file_path = "C:/Users/patha/OneDrive/Desktop/internships/vervebridge/task 3/cleaned_football_data.csv"
df.to_csv(cleaned_file_path, index=False)
print("Data has been cleaned and saved to:", cleaned_file_path)

Data has been cleaned and saved to: C:/Users/patha/OneDrive/Desktop/internships/vervebridge/task 3/cleaned_football_data.csv


In [21]:
cleaned_file_path = "C:/Users/patha/Documents/cleaned_football_data.csv"
df.to_csv(cleaned_file_path, index=False)